In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import StratifiedKFold
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tfe.enable_eager_execution()

split_num = 10


# data input 
path = 'trainrat_new.txt'
data = pd.read_csv(path,sep=' ')
print(len(data))

# Data extraction,Win
win_data = data[data.label == "W"]
win_data = win_data.assign(W=1,D=0,L=0) 
win_data = win_data.drop("label",axis=1)
win_data = win_data.sample(n=len(win_data))# random sort

# Data extraction,Draw
draw_data = data[data.label == "D"]
draw_data = draw_data.assign(W=0,D=1,L=0)
draw_data = draw_data.drop("label",axis=1)
draw_data = draw_data.sample(n=len(draw_data))# random sort

# Data extraction,Lose
lose_data = data[data.label == "L"]
lose_data = lose_data.assign(W=0,D=0,L=1) 
lose_data = lose_data.drop("label",axis=1)
lose_data = lose_data.sample(n=len(lose_data))# random sort

# Group data together,use for normal k-fold
all_data = pd.concat([win_data, draw_data, lose_data])
all_data = all_data.sample(n=len(all_data))# random sort
#yAll = xAll[["W","D","L"]]

# Data drop
#xWin = xWin.drop(["W","D","L"],axis=1)
#xDraw = xDraw.drop(["W","D","L"],axis=1)
#xLose = xLose.drop(["W","D","L"],axis=1)
#xAll = xAll.drop(["W","D","L"],axis=1)

# Data separate and making dataset
win_data_separate = []
draw_data_separate = []
lose_data_separate = []
all_data_separate = []
wdl_separate = []
for i in range(split_num):
    win_data_separate.append(win_data[i::split_num])
    draw_data_separate.append(draw_data[i::split_num])
    lose_data_separate.append(lose_data[i::split_num])
    all_data_separate.append(all_data[i::split_num])
    # merge for stratified sampling
    wdl_separate.append(pd.concat([win_data_separate[i],draw_data_separate[i],lose_data_separate[i]]))
    # assign a number to make final input data
    wdl_separate[i] = wdl_separate[i].assign(separate_num=i)
    all_data_separate[i] = all_data_separate[i].assign(separate_num=i)

# integrate everything once
wdl_separate_merge = wdl_separate[0]
all_data_separate_merge = all_data_separate[0]
for i in range(1,split_num):
    wdl_separate_merge = wdl_separate_merge.append(wdl_separate[i])
    all_data_separate_merge = all_data_separate_merge.append(all_data_separate[i])

#print(len(wdl_separate_merge))
#print(len(all_data_separate_merge))
    
    
# make final input data
x_train = []
y_train = []
x_test = []
y_test = []
xAll_train = []
yAll_train = []
xAll_test = []
yAll_test = []
for i in range(split_num):
    x_train.append(wdl_separate_merge[wdl_separate_merge['separate_num'] != i])
    x_test.append(wdl_separate_merge[wdl_separate_merge['separate_num'] == i])
    xAll_train.append(all_data_separate_merge[all_data_separate_merge['separate_num'] != i])
    xAll_test.append(all_data_separate_merge[all_data_separate_merge['separate_num'] == i])
for i in range(split_num):
    # delete separate_num
    x_train[i] = x_train[i].drop(['separate_num'],axis=1)
    x_test[i] = x_test[i].drop(['separate_num'],axis=1)
    xAll_train[i] = xAll_train[i].drop(['separate_num'],axis=1)
    xAll_test[i] = xAll_test[i].drop(['separate_num'],axis=1)
    # random sort
    x_train[i] = x_train[i].sample(n=len(x_train[i]))
    x_test[i] = x_test[i].sample(n=len(x_test[i]))
    xAll_train[i] = xAll_train[i].sample(n=len(xAll_train[i]))
    xAll_test[i] = xAll_test[i].sample(n=len(xAll_test[i]))
    
    # separate x and y
    y_train.append(x_train[i][["W","D","L"]])
    y_test.append(x_test[i][["W","D","L"]])
    yAll_train.append(xAll_train[i][["W","D","L"]])
    yAll_test.append(xAll_test[i][["W","D","L"]])
    x_train[i] = x_train[i].drop(["W","D","L"],axis=1)
    x_test[i] = x_test[i].drop(["W","D","L"],axis=1)
    xAll_train[i] = xAll_train[i].drop(["W","D","L"],axis=1)
    xAll_test[i] = xAll_test[i].drop(["W","D","L"],axis=1)


print(x_train[0].head())
print(y_train[0].head())
print(xAll_train[0].head())
print(yAll_test[0].head())


#print(len(y_train[0].query('W == 1')))
#print(len(y_train[0].query('L == 1')))
#print(len(y_train[0].query('D == 1')))
#print(len(yAll_train[0].query('W == 1')))
#print(len(yAll_train[0].query('L == 1')))
#print(len(yAll_train[0].query('D == 1')))




Using TensorFlow backend.


28186
          HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF     AAATT  \
16546  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
21026  0.086415  0.272678 -0.063278  0.361978  1.167127 -0.096290  0.133136   
18318  0.682548 -0.617906  0.406512 -0.073602 -0.223561  0.162641  0.096674   
21446  0.436292  0.042685  0.520847 -0.046465  0.679153 -0.156076  0.262545   
4863   0.551341 -0.149581 -0.061613  0.133302  0.097841  0.237495  0.109202   

          AADEF  
16546  0.000000  
21026 -0.077715  
18318  0.164822  
21446  0.007206  
4863   0.180779  
       W  D  L
16546  0  0  1
21026  0  1  0
18318  1  0  0
21446  0  1  0
4863   0  1  0
          HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF     AAATT  \
9996  -0.929910  0.337931 -0.044735  1.403678 -0.554042  0.207773 -0.237166   
26861  0.040592  0.307121  0.012601  0.232803  0.280745  0.397258  0.570748   
5209   0.572194  0.230175  0.296401  0.222982 -0.144833 -0.058910 -0.087241   
5727 

In [4]:
# create model
import tensorflow as tf
from keras import backend as K
modelArelu = Sequential()
modelArelu.add(Dense(10, activation='relu'))
modelArelu.add(Dense(10, activation='relu'))
modelArelu.add(Dense(3, activation='softmax'))

def MSE(y_true, y_pred):
    print(y_pred[0])
    print(y_true)
    print(y_pred-y_true)
    print(K.square(y_pred - y_true))
    return K.mean(K.square(y_pred - y_true), axis=-1)

def RPS(y_true, y_pred):
    print(y_true)
    #siz = tf.size(y_true)
    cumulative_sum_y_true = tf.Variable(0, tf.float32,name="cumulative_sum_y_true")
    cumulative_sum_y_pred = tf.Variable(0, tf.float32,name="cumulative_sum_y_pred")
    output = tf.Variable(0, tf.float32,name="output")
    for i in range(2):
        cumulative_sum_y_true = tf.add(cumulative_sum_y_true, y_true[i])
        cumulative_sum_y_pred = tf.add(cumulative_sum_y_true, y_pred[i])
        difference = tf.subtract(cumulative_sum_y_true,cumulative_sum_y_pred)
        difference2 = tf.scalar_mul(difference, difference)
        output = tf.add(output,difference2) 
        
    const1 = tf.constant(2)
    output = tf.divide(output,const1)
    return output


modelArelu.compile(loss=RPS,
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [5]:
cvscores = []
batch_size = 16
num_classes = 10
epochs = 10

for i in range(split_num):
    #print("TRAIN:", x_train, "TEST:", x_test)

    # Fit the model
    modelArelu.fit(x_train[i].values, y_train[i].values,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1
                  )


    results = modelArelu.predict(x_test[i], batch_size=batch_size, verbose=0, steps=None)
    
    print(results)
    print(y_test[i].values)


RuntimeError: tf.placeholder() is not compatible with eager execution.